Get code from github

In [1]:
! git clone https://github.com/balezz/bert4classification.git

fatal: destination path 'bert4classification' already exists and is not an empty directory.


In [2]:
%cd bert4classification

/content/bert4classification


Installing Packages

In [3]:
!pip install minio transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Download data

In [4]:
from minio import Minio

BUCKET_NAME = 'findatalake'
S3_FILE_NAME = 'rutwits.zip'
TMP_FILE = 'tmp.zip'
MINIO_URL = 'datalake.website:9000'

In [5]:
client = Minio(MINIO_URL,
               access_key='tester-1',
               secret_key='testerpass',
               secure=False)

In [6]:
client.fget_object(BUCKET_NAME, S3_FILE_NAME, TMP_FILE)

In [7]:
! unzip -a {TMP_FILE}

Archive:  tmp.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test.csv                [binary]
  inflating: train.csv               [binary]
  inflating: valid.csv               [binary]


And so one...

In [8]:
import pandas as pd

In [9]:
train_data = pd.read_csv('train.csv')
valid_data = pd.read_csv('valid.csv')
test_data  = pd.read_csv('test.csv')

In [10]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

Initialize BERT classifier

In [11]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=2
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Prepare data and helpers for train and evlauation

In [12]:
classifier.preparation(
        X_train=list(train_data['text']),
        y_train=list(train_data['label']),
        X_valid=list(valid_data['text']),
        y_valid=list(valid_data['label'])
    )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Train loop

In [ ]:
classifier.train()

Check test data

In [ ]:
texts = list(test_data['text'])
labels = list(test_data['label'])

predictions = [classifier.predict(t) for t in texts]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')